In [1]:
from keras.models import Sequential, load_model
from keras.layers import *
from keras import optimizers
from keras import utils
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
import keras

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

import tensorflow as tf

# Importing matplotlib to plot images.
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Importing SK-learn to calculate precision and recall
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneGroupOut
from sklearn.utils import shuffle 

# Used for graph export
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras import backend as K

import pickle as pkl
import h5py

from pathlib import Path
import os.path
import sys
import datetime
import time



Using TensorFlow backend.


In [2]:
import telegram
from keras.callbacks import Callback
from callbacks import TelegramCallback
from callbacks.TelegramData import TelegramData


# create callback
config = {
    'token': TelegramData.Token,   # paste your bot token
    'telegram_id': TelegramData.ID,                                   # paste your telegram_id
}

tg_callback = TelegramCallback(config)

In [3]:
class LoggingTensorBoard(TensorBoard):    

    def __init__(self, log_dir, settings_str_to_log, **kwargs):
        super(LoggingTensorBoard, self).__init__(log_dir, **kwargs)

        self.settings_str = settings_str_to_log

    def on_train_begin(self, logs=None):
        TensorBoard.on_train_begin(self, logs=logs)

        tensor =  tf.convert_to_tensor(self.settings_str)
        summary = tf.summary.text ("Run_Settings", tensor)

        with  tf.Session() as sess:
            s = sess.run(summary)
            self.writer.add_summary(s)

In [4]:
train_ids = [1,2,3, 7, 8, 9, 10,12,13,14,15,16]
test_ids = [4,5,6,11,17]

In [5]:
dfAll = pd.read_pickle("PklData/df_lstm.pkl")

In [8]:
df_train = dfAll[dfAll.userID.isin(train_ids)]
df_test = dfAll[dfAll.userID.isin(test_ids)]

df_train2 = df_train[['Blobs', 'TaskID']].copy()
df_test2 = df_test[['Blobs', 'TaskID']].copy()

In [10]:
x_train = np.vstack(df_train2.Blobs)
x_test = np.vstack(df_test2.Blobs)
y_train = df_train2.TaskID.values
y_test = df_test2.TaskID.values

x_train = x_train.reshape(-1, 27, 15, 1)
x_test = x_test.reshape(-1, 27, 15, 1)

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [11]:
df_train2.Blobs

0       [[[0, 224, 1, 0, 0, 161, 4, 2, 4, 3, 4, 0, 182...
1       [[[1, 224, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,...
2       [[[1, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 4, 2, 0, 0...
3       [[[0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0...
4       [[[0, 0, 0, 1, 1, 1, 5, 3, 0, 0, 0, 0, 0, 0, 0...
5       [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0...
6       [[[1, 0, 0, 0, 2, 1, 0, 0, 1, 1, 2, 0, 0, 0, 0...
7       [[[0, 0, 0, 1, 0, 0, 0, 0, 161, 0, 2, 7, 4, 1,...
8       [[[0, 0, 0, 182, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3,...
9       [[[0, 225, 1, 2, 0, 4, 5, 7, 1, 1, 0, 0, 0, 0,...
10      [[[2, 0, 0, 4, 1, 0, 1, 2, 1, 2, 1, 4, 5, 1, 0...
11      [[[0, 0, 0, 0, 0, 0, 0, 148, 0, 0, 174, 0, 0, ...
12      [[[0, 0, 0, 0, 165, 0, 0, 0, 163, 153, 6, 4, 2...
13      [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0...
14      [[[0, 2, 0, 2, 0, 1, 0, 1, 0, 0, 0, 0, 0, 2, 0...
15      [[[0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 174, 0, 0, 0,...
16      [[[3, 0, 1, 2, 0, 2, 1, 0, 0, 1, 171, 4, 2, 2,...
17      [[[178

In [ ]:
# If GPU is not available: 
# GPU_USE = '/cpu:0'
# config = tf.ConfigProto(device_count = {"GPU": 0})


# If GPU is available: 
config = tf.ConfigProto()
config.log_device_placement = True
config.allow_soft_placement = True
config.gpu_options.allocator_type = 'BFC'

# Limit the maximum memory used
config.gpu_options.per_process_gpu_memory_fraction = 0.1

# set session config
tf.keras.backend.set_session(tf.Session(config=config))

#Config for ConvLSTM

In [ ]:
tf.get_default_graph()
model = Sequential()
model.add(TimeDistributed(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'), input_shape=(None, train_x.shape[1], train_x.shape[2], 1)))
model.add(TimeDistributed(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu')))
model.add(TimeDistributed(MaxPool2D(pool_size=(2,2))))
model.add(TimeDistributed(Dropout(0.5)))

model.add(TimeDistributed(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu')))
model.add(TimeDistributed(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu')))
model.add(TimeDistributed(MaxPool2D(pool_size=(2,2))))
model.add(TimeDistributed(Dropout(0.5)))

model.add(TimeDistributed(Flatten()))

model.add(LSTM(75, return_sequences=True))
model.add(LSTM(75))
model.add(Dense(2))

optimizer = optimizers.Adam(lr = 0.001, decay=1e-6)
#optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.1)
model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])

# Broadcast progress to the tensorboard.
readable_timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d_%H%M%S')
tensorflowfolder = "/srv/share/tensorboardfiles/KnuckleFinger_LSTM_Jan_" + readable_timestamp

config = ""
for layer in model.layers:
    config += str(layer.output).split('\"')[1].split("/")[0] + str(layer.output_shape) + "\n\n"
config += "batchsize: " + str(batch_size) + "\n\n" + "epochs: " + str(epochs) + "\n\n"

model.summary()

readable_timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d_%H%M%S')
tensorflowfolder = "/srv/share/tensorboardfiles/KnuckleFinger_LSTM_Jan_" + readable_timestamp
print("KnuckleFinger_LSTM_Jan_" + readable_timestamp)
logger = LoggingTensorBoard(settings_str_to_log = config, log_dir=tensorflowfolder, histogram_freq=0,
                            write_graph=True, write_images=True, update_freq = 'epoch')
storer = ModelCheckpoint("./ModelSnapshots/KnuckleFinger_LSTM_Jan_" + readable_timestamp + ".h5", monitor='val_loss', verbose=0,
                         save_best_only=True, save_weights_only=False, mode='auto', period=1)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=10, 
                                            verbose=1, 
                                            factor=0.95, 
                                            min_lr=0.00001)

history = model.fit(x_train, y_train_one_hot,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test_one_hot),
                        callbacks=[storer,tg_callback, learning_rate_reduction])
